### Module 6: Dataframes and Spark SQL

#### Case Study: Mobile App Store
#### Domain: Telecom

In [ ]:
A telecom software provider is building an application to monitor different telecom 
components in the production environment. For monitoring purpose, the application 
relies on log files by parsing the log files and looking for potential warning or 
exceptions in the logs and reporting them.

The Dataset contains the log files from different components used in the overall 
telecom application.

Tasks:
The volume of data is quite large. As part of the R&D team, you are building a solution 
on spark to load and parse the multiple log files and then arranging the error and 
warning by the timestamp.
1. Load data into Spark DataFrame
2. Find out how many 404 HTTP codes are in access logs.
3. Find out which URLs are broken.
4. Verify there are no null columns in the original dataset.
5. Replace null values with constants such as 0
6. Parse timestamp to readable date.
7. Describe which HTTP status values appear in data and how many.
8. How many unique hosts are there in the entire log and their average request
9. Create a spark-submit application for the same and print the findings in the log

In [36]:
import findspark
findspark.init()

import os
from pyspark.sql import SparkSession, SQLContext

In [37]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import countDistinct, avg
from pyspark.sql.functions import dayofmonth, dayofyear, year, month, hour, weekofyear, date_format
from pyspark.sql.functions import col as func_col

In [38]:
#1. Load data into Spark DataFrame

In [39]:
spark = SparkSession.builder.appName("Module6CaseStudy2").getOrCreate()
df = df = spark.read.option("encoding", "windows-1252").csv('651_cs2_datasets_v1.0/access.log',inferSchema=True,sep=' ')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)



In [40]:
name_list = ['remote_host', '_c1', '_c2', 'request_time', 'request_zone', 'request_type', 'url', 'web_version', 'status',
 'bytes_setnt', '_c10', 'UA1', 'UA2', 'UA3', 'UA4', 'UA5', 'UA6', 'UA7', '_c18']
df = df.toDF(*name_list)


In [41]:
from pyspark.sql.functions import concat,concat_ws
df = df.select("remote_host",
               '_c1',
               '_c2',
          concat_ws(' ',df['request_time'],df['request_zone']).alias('Time'),
          df['request_type'].alias('Type'),
         df['url'].alias('URL'),
         df['web_version'].alias('Web'),
         df['status'],
         df['bytes_setnt'].alias('Bytes'),
               '_c10',
         concat_ws(' ','UA1','UA2','UA3','UA4','UA5','UA6','UA7').alias('UserAgent'),
              '_c18')


In [42]:
df = df.replace('-',None,['_c1','_c2','_c10','_c18'])

In [46]:
spark.catalog.dropTempView('Access')

True

In [47]:
df.createTempView('Access')

In [48]:
spark.sql("select * from Access limit 10").show()

+---------------+----+----+--------------------+----+--------------------+--------+------+-----+--------------------+--------------------+----+
|    remote_host| _c1| _c2|                Time|Type|                 URL|     Web|status|Bytes|                _c10|           UserAgent|_c18|
+---------------+----+----+--------------------+----+--------------------+--------+------+-----+--------------------+--------------------+----+
|109.169.248.247|null|null|[12/Dec/2015:18:2...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
|109.169.248.247|null|null|[12/Dec/2015:18:2...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind...|null|
|    46.72.177.4|null|null|[12/Dec/2015:18:3...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
|    46.72.177.4|null|null|[12/Dec/2015:18:3...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind..

In [ ]:
# 2. Find out how many 404 HTTP codes are in access logs.

In [49]:
spark.sql("select count(status) from Access where status='404'").show()

+-------------+
|count(status)|
+-------------+
|       227089|
+-------------+



In [50]:
# 3. Find out which URLs are broken.

In [51]:
spark.sql("select count(url) from Access where length(url)=0").show()

+----------+
|count(url)|
+----------+
|         0|
+----------+



In [ ]:
# 4. Verify there are no null columns in the original dataset.

In [14]:
select count(isnan())

DataFrame[summary: string, remote_host: string, _c1: string, _c2: string, Time: string, Type: string, URL: string, Web: string, status: string, Bytes: string, _c10: string, UserAgent: string, _c18: string]

In [60]:
from pyspark.sql.functions import col

spark.sql("""select sum(if(remote_host is Null, 1,0)) as remote_host,
                  sum(if(_c1 is Null, 1,0)) _c1,
                  sum(if(_c2 is Null, 1,0)) _c2,
                  sum(if(Time is Null, 1,0)) Time,
                  sum(if(Type is Null, 1,0)) Type,
                  sum(if(URL is Null, 1,0)) URL,
                  sum(if(Web is Null, 1,0)) Web,
                  sum(if(status is Null, 1,0)) status,
                  sum(if(Bytes is Null, 1,0)) Bytes,
                  sum(if(_c10 is Null, 1,0)) _c10,
                  sum(if(UserAgent is Null, 1,0)) UserAgent,
                  sum(if(_c18 is Null, 1,0)) _c18
          from Access""").show()

+-----------+-------+-------+----+----+---+---+------+-----+------+---------+------+
|remote_host|    _c1|    _c2|Time|Type|URL|Web|status|Bytes|  _c10|UserAgent|  _c18|
+-----------+-------+-------+----+----+---+---+------+-----+------+---------+------+
|          0|2338006|2337872|   0|   0|  0| 13|     0|    0|677871|        0|796604|
+-----------+-------+-------+----+----+---+---+------+-----+------+---------+------+



In [ ]:
# 5. Replace null values with constants such as 0

In [62]:
df = df.fillna({'_c1':0,'_c2':0, '_c10':0, '_c18':0, 'web':0})

In [64]:
spark.catalog.dropTempView('Access')

True

In [65]:
df.createTempView('Access')

In [66]:
spark.sql("""select sum(if(remote_host is Null, 1,0)) as remote_host,
                  sum(if(_c1 is Null, 1,0)) _c1,
                  sum(if(_c2 is Null, 1,0)) _c2,
                  sum(if(Time is Null, 1,0)) Time,
                  sum(if(Type is Null, 1,0)) Type,
                  sum(if(URL is Null, 1,0)) URL,
                  sum(if(Web is Null, 1,0)) Web,
                  sum(if(status is Null, 1,0)) status,
                  sum(if(Bytes is Null, 1,0)) Bytes,
                  sum(if(_c10 is Null, 1,0)) _c10,
                  sum(if(UserAgent is Null, 1,0)) UserAgent,
                  sum(if(_c18 is Null, 1,0)) _c18
          from Access""").show()

+-----------+---+---+----+----+---+---+------+-----+----+---------+----+
|remote_host|_c1|_c2|Time|Type|URL|Web|status|Bytes|_c10|UserAgent|_c18|
+-----------+---+---+----+----+---+---+------+-----+----+---------+----+
|          0|  0|  0|   0|   0|  0|  0|     0|    0|   0|        0|   0|
+-----------+---+---+----+----+---+---+------+-----+----+---------+----+



In [ ]:
# 6. Parse timestamp to readable date.

In [82]:
spark.sql("select to_timestamp(substring(Time,2,26) ,'dd/MMM/yyyy:HH:mm:ss x') Date_Parsed from Access").show(truncate=False)

+-------------------+
|Date_Parsed        |
+-------------------+
|2015-12-12 18:25:11|
|2015-12-12 18:25:11|
|2015-12-12 18:31:08|
|2015-12-12 18:31:08|
|2015-12-12 18:31:25|
|2015-12-12 18:31:25|
|2015-12-12 18:32:10|
|2015-12-12 18:32:11|
|2015-12-12 18:32:56|
|2015-12-12 18:32:56|
|2015-12-12 18:33:51|
|2015-12-12 18:33:52|
|2015-12-12 18:36:16|
|2015-12-12 18:36:16|
|2015-12-12 18:38:42|
|2015-12-12 18:38:42|
|2015-12-12 18:38:55|
|2015-12-12 18:38:56|
|2015-12-12 18:39:27|
|2015-12-12 18:39:27|
+-------------------+
only showing top 20 rows



In [ ]:
# 7. Describe which HTTP status values appear in data and how many.

In [85]:
spark.sql("SELECT status,count(status) from Access group by status").show()

+--------+-------------+
|  status|count(status)|
+--------+-------------+
|     200|      1157831|
|     206|       939929|
|     501|          143|
|     404|       227089|
|     500|         3252|
|     304|         6330|
|     405|           83|
|     301|          619|
|     403|         2222|
|     412|           19|
|     303|          247|
|     400|           23|
|     401|          153|
|HTTP/1.1|           13|
|     406|           53|
+--------+-------------+



In [ ]:
# 8. How many unique hosts are there in the entire log and their average request

In [87]:
spark.sql("select count(distinct(remote_host)) from Access").show()

+---------------------------+
|count(DISTINCT remote_host)|
+---------------------------+
|                      40836|
+---------------------------+



In [94]:
spark.sql("select avg(cnt) from (select remote_host,count(remote_host) cnt from Access  group by remote_host) t").show()

+-----------------+
|         avg(cnt)|
+-----------------+
|57.25355078851993|
+-----------------+



In [95]:
spark.sql("select count(*) from Access").show()

+--------+
|count(1)|
+--------+
| 2338006|
+--------+



In [96]:
2338006/40836

57.25355078851993

In [ ]:
# 9. Create a spark-submit application for the same and print the findings in the log

In [ ]:
C:\Users\suzuk\Documents\edureka\PySpark Certification Training Course\module 6>spark-submit Module6_Case_Study2_Saravanan.py
****************************************************************************************************
1. Load data into Spark DataFrame
****************************************************************************************************
23/02/25 15:16:39 INFO SparkContext: Running Spark version 3.3.2
23/02/25 15:16:39 INFO ResourceUtils: ==============================================================
23/02/25 15:16:39 INFO ResourceUtils: No custom resources configured for spark.driver.
23/02/25 15:16:39 INFO ResourceUtils: ==============================================================
23/02/25 15:16:39 INFO SparkContext: Submitted application: Module6CaseStudy2
23/02/25 15:16:39 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
23/02/25 15:16:39 INFO ResourceProfile: Limiting resource is cpu
23/02/25 15:16:39 INFO ResourceProfileManager: Added ResourceProfile id: 0
23/02/25 15:16:39 INFO SecurityManager: Changing view acls to: suzuk
23/02/25 15:16:39 INFO SecurityManager: Changing modify acls to: suzuk
23/02/25 15:16:39 INFO SecurityManager: Changing view acls groups to:
23/02/25 15:16:39 INFO SecurityManager: Changing modify acls groups to:
23/02/25 15:16:39 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(suzuk); groups with view permissions: Set(); users  with modify permissions: Set(suzuk); groups with modify permissions: Set()
23/02/25 15:16:40 INFO Utils: Successfully started service 'sparkDriver' on port 63716.
23/02/25 15:16:40 INFO SparkEnv: Registering MapOutputTracker
23/02/25 15:16:40 INFO SparkEnv: Registering BlockManagerMaster
23/02/25 15:16:40 INFO BlockManagerMasterEndpoint: Using org.apache.spark.storage.DefaultTopologyMapper for getting topology information
23/02/25 15:16:40 INFO BlockManagerMasterEndpoint: BlockManagerMasterEndpoint up
23/02/25 15:16:40 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/02/25 15:16:40 INFO DiskBlockManager: Created local directory at C:\Users\suzuk\AppData\Local\Temp\blockmgr-23826b14-5708-450c-a96f-e802140c34be
23/02/25 15:16:40 INFO MemoryStore: MemoryStore started with capacity 366.3 MiB
23/02/25 15:16:40 INFO SparkEnv: Registering OutputCommitCoordinator
23/02/25 15:16:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/25 15:16:41 INFO Utils: Successfully started service 'SparkUI' on port 4041.
23/02/25 15:16:41 INFO Executor: Starting executor ID driver on host LAPTOP-TCJS4952
23/02/25 15:16:41 INFO Executor: Starting executor with user classpath (userClassPathFirst = false): ''
23/02/25 15:16:41 INFO Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 63767.
23/02/25 15:16:41 INFO NettyBlockTransferService: Server created on LAPTOP-TCJS4952:63767
23/02/25 15:16:41 INFO BlockManager: Using org.apache.spark.storage.RandomBlockReplicationPolicy for block replication policy
23/02/25 15:16:41 INFO BlockManagerMaster: Registering BlockManager BlockManagerId(driver, LAPTOP-TCJS4952, 63767, None)
23/02/25 15:16:41 INFO BlockManagerMasterEndpoint: Registering block manager LAPTOP-TCJS4952:63767 with 366.3 MiB RAM, BlockManagerId(driver, LAPTOP-TCJS4952, 63767, None)
23/02/25 15:16:41 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, LAPTOP-TCJS4952, 63767, None)
23/02/25 15:16:41 INFO BlockManager: Initialized BlockManager: BlockManagerId(driver, LAPTOP-TCJS4952, 63767, None)
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)

+---------------+----+----+--------------------+----+--------------------+--------+------+-----+--------------------+--------------------+----+
|    remote_host| _c1| _c2|                Time|Type|                 URL|     Web|status|Bytes|                _c10|           UserAgent|_c18|
+---------------+----+----+--------------------+----+--------------------+--------+------+-----+--------------------+--------------------+----+
|109.169.248.247|null|null|[12/Dec/2015:18:2...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
|109.169.248.247|null|null|[12/Dec/2015:18:2...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind...|null|
|    46.72.177.4|null|null|[12/Dec/2015:18:3...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
|    46.72.177.4|null|null|[12/Dec/2015:18:3...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind...|null|
| 83.167.113.100|null|null|[12/Dec/2015:18:3...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
| 83.167.113.100|null|null|[12/Dec/2015:18:3...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind...|null|
|   95.29.198.15|null|null|[12/Dec/2015:18:3...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
|   95.29.198.15|null|null|[12/Dec/2015:18:3...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind...|null|
|  109.184.11.34|null|null|[12/Dec/2015:18:3...| GET|     /administrator/|HTTP/1.1|   200| 4263|                null|Mozilla/5.0 (Wind...|null|
|  109.184.11.34|null|null|[12/Dec/2015:18:3...|POST|/administrator/in...|HTTP/1.1|   200| 4494|http://almhuette-...|Mozilla/5.0 (Wind...|null|
+---------------+----+----+--------------------+----+--------------------+--------+------+-----+--------------------+--------------------+----+


****************************************************************************************************
2. Find out how many 404 HTTP codes are in access logs.
****************************************************************************************************
+-------------+
|count(status)|
+-------------+
|       227089|
+-------------+


****************************************************************************************************
3. Find out which URLs are broken.
****************************************************************************************************
+----------+
|count(url)|
+----------+
|         0|
+----------+


****************************************************************************************************
4. Verify there are no null columns in the original dataset.
****************************************************************************************************
+-----------+-------+-------+----+----+---+---+------+-----+------+---------+------+
|remote_host|    _c1|    _c2|Time|Type|URL|Web|status|Bytes|  _c10|UserAgent|  _c18|
+-----------+-------+-------+----+----+---+---+------+-----+------+---------+------+
|          0|2338006|2337872|   0|   0|  0| 13|     0|    0|677871|        0|796604|
+-----------+-------+-------+----+----+---+---+------+-----+------+---------+------+


****************************************************************************************************
5. Replace null values with constants such as 0
****************************************************************************************************
+-----------+---+---+----+----+---+---+------+-----+----+---------+----+
|remote_host|_c1|_c2|Time|Type|URL|Web|status|Bytes|_c10|UserAgent|_c18|
+-----------+---+---+----+----+---+---+------+-----+----+---------+----+
|          0|  0|  0|   0|   0|  0|  0|     0|    0|   0|        0|   0|
+-----------+---+---+----+----+---+---+------+-----+----+---------+----+


****************************************************************************************************
6. Parse timestamp to readable date.
****************************************************************************************************
+-------------------+
|Date_Parsed        |
+-------------------+
|2015-12-12 18:25:11|
|2015-12-12 18:25:11|
|2015-12-12 18:31:08|
|2015-12-12 18:31:08|
|2015-12-12 18:31:25|
|2015-12-12 18:31:25|
|2015-12-12 18:32:10|
|2015-12-12 18:32:11|
|2015-12-12 18:32:56|
|2015-12-12 18:32:56|
|2015-12-12 18:33:51|
|2015-12-12 18:33:52|
|2015-12-12 18:36:16|
|2015-12-12 18:36:16|
|2015-12-12 18:38:42|
|2015-12-12 18:38:42|
|2015-12-12 18:38:55|
|2015-12-12 18:38:56|
|2015-12-12 18:39:27|
|2015-12-12 18:39:27|
+-------------------+
only showing top 20 rows


****************************************************************************************************
7. Describe which HTTP status values appear in data and how many.
****************************************************************************************************
+--------+-------------+
|  status|count(status)|
+--------+-------------+
|     200|      1157831|
|     206|       939929|
|     501|          143|
|     404|       227089|
|     500|         3252|
|     304|         6330|
|     405|           83|
|     301|          619|
|     403|         2222|
|     412|           19|
|     303|          247|
|     400|           23|
|     401|          153|
|HTTP/1.1|           13|
|     406|           53|
+--------+-------------+


****************************************************************************************************
8. How many unique hosts are there in the entire log and their average request
****************************************************************************************************
+---------------------------+
|count(DISTINCT remote_host)|
+---------------------------+
|                      40836|
+---------------------------+

+-----------------+
|         avg(cnt)|
+-----------------+
|57.25355078851993|
+-----------------+

+--------+
|count(1)|
+--------+
| 2338006|
+--------+

57.25355078851993